In [10]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from scipy.sparse import hstack
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import punkt

In [20]:
train = pd.read_csv('cleaned_tweets.csv', index_col=0)

sources_not_to_keep = ['dataset2_abhiraj', 'dataset3_abhiraj', 'dataset1_abhiraj']

print(train.shape)
train = train[train.Cleaned != 'None']
train = train[~train.Source.isin(sources_not_to_keep)]
train.dropna(axis = 0, inplace = True)
print(train.shape)

train= train.sample(frac=1).reset_index(drop=True)

(28992, 5)
(16441, 5)


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['Cleaned_stop_words'], train['Target'], test_size=0.20, random_state=42)

In [22]:
stop_words = stopwords.words('english')

embeddings_index = {}
f = open('glove.twitter.27B.100d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
       coefs = np.asarray(values[1:], dtype='float32')
       embeddings_index[word] = coefs
    except ValueError:
       pass
f.close()

def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())

xtrain_glove = [sent2vec(x) for x in tqdm(X_train)]
xtest_glove = [sent2vec(x) for x in tqdm(X_test)]

print('Checkpoint2 -Normalized Vector for Sentences are created')

xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)

3727it [00:00, 37266.84it/s]

Checkpoint1 - Data Read Complete


1193514it [00:29, 40971.73it/s]
  6%|▌         | 734/13152 [00:00<00:01, 7334.46it/s]

Found 1193514 word vectors.


100%|██████████| 3289/3289 [00:00<00:00, 7206.34it/s]

Checkpoint2 -Normalized Vector for Sentences are created


In [23]:
train_target = train
classifier = LogisticRegression(solver='lbfgs')
classifier.fit(xtrain_glove, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
print('Training LogisticRegression Classifier is complete!!')
predictions = classifier.predict(xtest_glove)

Training LogisticRegression Classifier is complete!!


In [25]:
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.87      0.89      0.88      2235
         1.0       0.75      0.73      0.74      1054

    accuracy                           0.83      3289
   macro avg       0.81      0.81      0.81      3289
weighted avg       0.83      0.83      0.83      3289

